<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/CravingsBuffet_ElaboraFotoRandom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install clarifai

In [0]:
#Da cambiare:
#nome del csv e cartella da cui prendo le foto (foto ordinate come "ultima modifica e freccetta in giù")

import json
import csv
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive

# apro e parso il file .json
def convert(response,file):

  with open('utenti.txt', 'w') as outfile:
    json.dump(response, outfile) 
        
  # apro e parso il file .json
  data = open('utenti.txt').read()
  data_parsed = json.loads(data)

  #creo e apro un file .csv per scrivere
  file_parsed = open('CravingsBuffet_10%Random_UsersData.csv', 'a')
    
  #creo il writer
  writer = csv.writer(file_parsed)

  #ciclo per estrarre dal file json i valori di id, età, genere ed etnia
  for person in data_parsed['user']:
     writer.writerow([person['id'],
                      person['age'],
                      person['gender'],
                      person['ethnicity']])
     
  
  #salvo e chiudo il file .csv
  file_parsed.close()

  file.SetContentFile('CravingsBuffet_10%Random_UsersData.csv') #The contents of the file
  file.Upload()

In [0]:
from clarifai.rest import ClarifaiApp
import time

def detect_clarifai(id, url_photo, data):
  app = ClarifaiApp(api_key='018868a318714c1a83b53d17b1319e14')
  model = app.models.get(model_id="c0c0ac362b03416da06ab3fa36fb58e3")
  response = model.predict_by_url(url = '%s' % url_photo)
  
  if response['outputs'][0]['data'] != {}:
    x = 0
    while (x < len(response['outputs'][0]['data']['regions'])):
      age = response['outputs'][0]['data']['regions'][x]['data']['concepts'][0]['name']
      gender = response['outputs'][0]['data']['regions'][x]['data']['concepts'][20]['name']
      ethnicity = response['outputs'][0]['data']['regions'][x]['data']['concepts'][22]['name']

      data['user'].append({
          'id' : '%s' % id,
          'age' : '%s' % age,
          'gender' : '%s' % gender,
          'ethnicity' : '%s' % ethnicity,
      })
      x = x + 1
  else:
    return

In [22]:
#Da cambiare:
#id_csv_clarifai = link condivisibile del csv (clarifai) già fatto e il nome
#file_list = cartella da dove prendi le foto

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
from clarifai.rest import ApiError
import json
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# DOWNLOAD my_users.csv FROM DRIVE
id_csv_clarifai = '12aiPnkvuUtyc8B2UmJEu2BpeXVYbPumF'  # FILE ID, got on google drive with condivision link 
download = drive.CreateFile({'id': id_csv_clarifai})
download.GetContentFile('CravingsBuffet_10%Random_UsersData.csv')

#leggo il csv
csv_clarifai = pd.read_csv('CravingsBuffet_10%Random_UsersData.csv')

#Auto-iterate through all files that matches this query
file_list = drive.ListFile({'q': "'1xeUzi-MjYk4-mVCG5QKVZkGed0uy9BWR' in parents and trashed=false"}).GetList()    #cartella da cui prendo le foto

data = {}
data['user'] = []
for file1 in file_list:
  id = file1['title'][0:22] 
  df1 = csv_clarifai[csv_clarifai['id'].str.contains(id)]
  data = {}
  data['user'] = []
  if df1.empty :
    detect_clarifai(id, file1['thumbnailLink'], data)
    convert(data, download)


ApiError: ignored